In [8]:
import assemblyai as aai
from pydantic import BaseModel
from openai import OpenAI
import json
from datetime import datetime
import locale
import re
import os  
from dotenv import load_dotenv

load_dotenv()

from prompts import SYSTEM_PROMPT, CLINICAL_NOTE_EXAMPLE
from utils import (deduplicate_tags,
    get_text_from_transcript,
    generate_temporal_context,
    count_tokens,
    save_transcription,
    get_transcription,
    save_clinical_note,
    get_clinical_note,
    get_audio_length
    )

In [9]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

ASSEMBLY_KEY = os.getenv("ASSEMBLY_KEY")
aai.settings.api_key = ASSEMBLY_KEY

AUDIO_PATH = "data/audios/"
TRANSCRIPTIONS_PATH = "data/transcriptions/"
CLINICAL_NOTES_PATH = "data/clinical_notes/"

audio_file = "005.m4a"

In [10]:
def transcribe_audio(audio_file, reload=False):
  audio_file_basename, extension = os.path.splitext(os.path.basename(audio_file))
  transcription_path = os.path.join(TRANSCRIPTIONS_PATH, audio_file_basename + ".txt")
  audio_path = os.path.join(AUDIO_PATH, audio_file)
  #Load transcription if exists and reload is False
  if not reload:
    transcription = get_transcription(transcription_path)
    if transcription:
      return transcription

  config = aai.TranscriptionConfig(
    speech_model=aai.SpeechModel.universal,
    speaker_labels=True,
    language_code="es",
    speakers_expected=2,
  ).set_redact_pii(
      policies=[
          aai.PIIRedactionPolicy.person_name,
          aai.PIIRedactionPolicy.email_address,
          aai.PIIRedactionPolicy.phone_number,
          aai.PIIRedactionPolicy.us_social_security_number,
      ],
      substitution=aai.PIISubstitutionPolicy.entity_name,
  )
  transcript = aai.Transcriber(config=config).transcribe(audio_path)
  text = get_text_from_transcript(transcript)
  save_transcription(text, transcription_path)
  return text

In [11]:
transcription = transcribe_audio(audio_file, reload=False)

In [12]:
def generate_clinical_note(user_prompt, system_prompt, audio_file, reload = False):
    audio_file_basename, _ = os.path.splitext(os.path.basename(audio_file))
    clinical_note_filename = os.path.join(CLINICAL_NOTES_PATH, audio_file_basename + ".json")
    if not reload:
        clinical_note = get_clinical_note(clinical_note_filename)
        if clinical_note:
            return clinical_note

    completion = client.responses.create(
        model="gpt-5",
        reasoning={"effort": "minimal"},
        input=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": (
                    user_prompt
                ),
            },
        ],
        text={"format": {"type": "json_object"}},
    )
    data = json.loads(completion.output[1].content[0].text)
    save_clinical_note(data, clinical_note_filename)
    return data

In [13]:
temporal_context = generate_temporal_context()
system_prompt = SYSTEM_PROMPT.format(temporal_context=temporal_context, clinical_note_example=CLINICAL_NOTE_EXAMPLE)

clinical_note = generate_clinical_note(transcription, system_prompt, audio_file, reload=False)
clinical_note_text = json.dumps(clinical_note, indent=4, ensure_ascii=False)

In [14]:
audio_path = os.path.join(AUDIO_PATH, audio_file)
audio_time = get_audio_length(audio_path)
input_tokens = count_tokens(transcription+system_prompt)
output_tokens = count_tokens(clinical_note_text)

dolar_a_cop = 4000

cost_input = input_tokens * (1.25/1000000) * dolar_a_cop
cost_output = output_tokens * (10/1000000) * dolar_a_cop
cost_transcription = audio_time * (0.15/3600) * dolar_a_cop
cost_pii_redaction = audio_time * (0.08/3600) * dolar_a_cop

print(f"Costo de entrada GPT: {cost_input} COP")
print(f"Costo de salida GPT: {cost_output} COP")
print(f"Costo total GPT: {cost_input + cost_output} COP")
print("-"*50)

print(f"Costo de transcripción: {cost_transcription} COP")
print(f"Costo de redacción PII: {cost_pii_redaction} COP")
print(f"Costo total transcripción: {cost_transcription + cost_pii_redaction} COP")
print("-"*50)

print(f"COSTO TOTAL: {cost_input + cost_output + cost_transcription + cost_pii_redaction} COP")

Costo de entrada GPT: 26.540000000000003 COP
Costo de salida GPT: 48.36000000000001 COP
Costo total GPT: 74.9 COP
--------------------------------------------------
Costo de transcripción: 140.49066666666664 COP
Costo de redacción PII: 74.92835555555556 COP
Costo total transcripción: 215.4190222222222 COP
--------------------------------------------------
COSTO TOTAL: 290.3190222222222 COP
